## Classify_df

### Useful links

#### <u>Datasets</u>
Complete dataset: The full dataset of images used can be found [here](https://drive.google.com/drive/folders/1Rz0JrjUCU-4VmDkolbkwUcM8xW1jd9pl?usp=drive_link)

Cropped datasets: The complete dataset has been used by SAM to create segmentations of cells, with some noise, resulting in:
 All these [crops](https://drive.google.com/drive/folders/1Rz0JrjUCU-4VmDkolbkwUcM8xW1jd9pl?usp=drive_link) 
 and all these [csv files](https://frbautneduar-my.sharepoint.com/:u:/g/personal/ntaurozzi_frba_utn_edu_ar/EYKi5F-wXGRNkAqjPSRVhvUByTsnsEB10OrZiJHclkOPWQ?e=tVwmkS) with the information of each crop.

Input/Target dataset: From the complete dataset, some images have been tagged by the biologists an those 58 (for now) can be found [here](https://frbautneduar-my.sharepoint.com/:u:/g/personal/ntaurozzi_frba_utn_edu_ar/EQbvUOwADihJsJLAyVfBdYwBDvHJDMS5GQuyyP_PzUeCLQ?e=z8A7Tn) each image with its corresponding target. The name of the images here are ids given to them by a json file.

#### <u>Auxiliary files</u>
To create the input/target dataset from the complete dataset, this [json](https://drive.google.com/file/d/1ydQ2fIOllwPPU64Kneda4mVidUww1X9T/view?usp=drive_link) was used

#### <u>Models</u>
For making the predictions these [models](https://frbautneduar-my.sharepoint.com/:f:/g/personal/lmareque_frba_utn_edu_ar/EiDo8WYptOpEiyzJHhQIbwoBUAfsoULwRRKEm-fmgzQD-g?e=6TE9yu) will be used

### Code walkthrough

This notebook will process all the images from the input/target dataset. It will use the json file to map the IDs to their original filenames. Then, it will search for all the crops belonging to these images and use all the models to predict whether each crop is noise or a cell. After the prediction, it will store the results in the corresponding CSV file for each image.

The output will be a folder for each model used. Within each folder, a CSV file will be created for each image, containing information about the bounding boxes of each crop and its classification (cell or noise).

### Imports

In [1]:
import pandas as pd
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
import joblib
import math
import keras
import cv2 as cv
import json
sys.path.insert(0, "../packages/python")

2024-12-26 10:45:53.939409: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-26 10:45:53.946529: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-26 10:45:53.954583: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-26 10:45:53.956858: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-26 10:45:53.963107: I tensorflow/core/platform/cpu_feature_guar

### Paths

In [13]:
CSV_PATH = '../output/sam_uploaded_out/'
CROPS_PATH = '../output/cropped_cells_full/'
MODELS_PATH = '../models/'
IMAGES_PATH = '../media/data/input/'
JSON_PATH = '../media/corte-28-10-2023.json'
OUTPUT_PATH = 

# CSV_PATH = '../untracked/datasets/Abril2023/sam_uploaded_out'
# CROPS_PATH = '../untracked/datasets/Abril2023/cropped_cells'
# MODELS_PATH = '../untracked/test models'
# IMAGES_PATH = '../untracked/datasets/Abril2023/raw_dataset/input'
# JSON_PATH = '../untracked/datasets/Abril2023/corte-28-10-2023.json'

# OUTPUT_PATH = '../untracked/outputs'

### Functions

In [3]:
def predict_cell(model_path, image_path, images_batch):
  """
  Given an image batch it returns the predictions of the batch with the given model.

  Args:
    model_path: path to the keras model to use.
    image_path: path to the folder where the images are.
    images_batch: list of the image names to include in the batch

  Returns:
    A list of predictions.
  """
  images = []
  for image in images_batch:
      img = cv.imread(os.path.join(image_path,image))#, cv.IMREAD_GRAYSCALE)
      img = cv.resize(img, (128, 128))
      img = img / 255.0 
      images.append(img)

  model = keras.models.load_model(model_path)
  batch = np.stack(images)
  prediction = model.predict(batch, verbose=0)

  return prediction #True if prediction >= 0.5 else False


def find_image_name(data, id):
  """
  Given a list of objects it searches the file_name from the id.

  Args:
    data: list of objects to search from.
    id: id of the file name to return.

  Returns:
    The file name of the image id.
  """
  for img in data:
      if img['id'] == id:
          base_name, _ = os.path.splitext(img['file_name'])
          return base_name
        

def process_images_in_batches(string_list, batch_size=10):
  """
  Processes a list of strings in batches of a specified size.

  Args:
    string_list: The list of strings to process.
    batch_size: The size of each batch.

  Yields:
    A batch of strings.
  """
  for i in range(0, len(string_list), batch_size):
    batch_num = i // batch_size + 1  # Calculate batch number (1-indexed)
    yield batch_num, string_list[i:i + batch_size]

### Lists of elements to use

In [4]:
csvs = sorted(os.listdir(CSV_PATH)) #Paths to the csv of SAM detections of each image
crops = sorted(os.listdir(CROPS_PATH)) #Paths to the crops made from SAM detection of the full_images
models = sorted(os.listdir(MODELS_PATH)) #Models to use in the prediction
og_images = sorted(os.listdir(IMAGES_PATH)) #full_images from where the crops are made
with open(JSON_PATH, 'r') as f: #json with the information of the filename of the images
    data = json.load(f)

### Model prediction

In [5]:
for model_idx, model in enumerate(models):

    base_model, ext = os.path.splitext(model)
    if '.keras' != ext:
        continue

    output = os.path.join(OUTPUT_PATH, "evaluated", base_model)

    if not os.path.exists(output): #Create dirs for each model used
        os.makedirs(output)

    for og_image_idx, og_image in enumerate(og_images):
        img_name, _ = os.path.splitext(og_image)
        real_name = find_image_name(data['images'], int(img_name)) #Find the real image name from where the crops where made
        images = sorted([crop for crop in crops if crop.startswith(real_name)]) #Get all the crops from that image
        df = pd.read_csv(os.path.join(CSV_PATH, f"{real_name}.csv")) #Read the csv of that image

        batch_size=30
        for idx, batch in process_images_in_batches(images, batch_size=batch_size): #Read the images in batch_size batches
            print(f"Model: {model} ({model_idx+1}/{len(models)}) - Image: {og_image} ({og_image_idx+1}/{len(og_images)}) - Batch {idx}/{math.ceil(len(images)/batch_size)}", end='\r')

            batch_prediction = predict_cell(model_path=os.path.join(MODELS_PATH, model), image_path=CROPS_PATH, images_batch=batch)
            is_cell = [True if sublist[0] >= 0.5 else False for sublist in batch_prediction]
            # print(is_cell)

            for idx, i in enumerate(batch):#For each batch store the prediction in the csv
                crop_name, _ = os.path.splitext(i)
                cell_id = crop_name.split('_')[2]

                mask = (df['cell_id'] == int(cell_id)) & (df['image'] == real_name)
                df.loc[mask, 'is_cell'] = is_cell[idx]
                df.loc[mask, 'image'] = img_name 
        df.to_csv(os.path.join(output, f"{img_name}.csv"))

I0000 00:00:1735220755.007641  150891 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1735220755.007793  150891 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1735220755.007904  150891 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1735220755.008017  150891 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

W0000 00:00:1735220760.406753  151074 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220760.407322  151074 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220760.407828  151074 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220760.408310  151074 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220760.408814  151074 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220760.409362  151074 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220760.409975  151074 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220760.410492  151074 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220760.411016  151074 gp

W0000 00:00:1735220760.607878  151074 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220760.608624  151074 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220760.609393  151074 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220760.610155  151074 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220760.610906  151074 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220760.611657  151074 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220760.612517  151074 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220760.613298  151074 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220760.614060  151074 gp

W0000 00:00:1735220765.604096  151074 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220765.604752  151074 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220765.605374  151074 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220765.605959  151074 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220765.606550  151074 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220765.607262  151074 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220765.608109  151074 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220765.608803  151074 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220765.609484  151074 gp

W0000 00:00:1735220769.119844  151067 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220769.120560  151067 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220769.121189  151067 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220769.121817  151067 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220769.122441  151067 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220769.123218  151067 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220769.124159  151067 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220769.124901  151067 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220769.125633  151067 gp

...

W0000 00:00:1735220914.385997  151079 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220914.386731  151079 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220914.387393  151079 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220914.388040  151079 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220914.388691  151079 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220914.389482  151079 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220914.390456  151079 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220914.391209  151079 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220914.391957  151079 gp

W0000 00:00:1735220921.867729  151076 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220921.868178  151076 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220921.868572  151076 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220921.868949  151076 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220921.869328  151076 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220921.869741  151076 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220921.870190  151076 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220921.870633  151076 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220921.871048  151076 gp

W0000 00:00:1735220941.680477  151072 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220941.681089  151072 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220941.681641  151072 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220941.682184  151072 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220941.682726  151072 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220941.683355  151072 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220941.684089  151072 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220941.684703  151072 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220941.685322  151072 gp

W0000 00:00:1735220949.433480  151072 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220949.434019  151072 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220949.434509  151072 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220949.434982  151072 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220949.435467  151072 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220949.436017  151072 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220949.436593  151072 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220949.437133  151072 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220949.437654  151072 gp

W0000 00:00:1735220961.539053  151065 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220961.539544  151065 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220961.539992  151065 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220961.540420  151065 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220961.540850  151065 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220961.541323  151065 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220961.541815  151065 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220961.542270  151065 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1735220961.542723  151065 gp

# Results

In [14]:
for model_idx, model in enumerate(models):
    output = os.path.join(OUTPUT_PATH, "evaluated", base_model)
    if not os.path.exists(output):
        continue
    for og_image_idx, og_image in enumerate(og_images):
        df = pd.read_csv(os.path.join(output, f"{img_name}.csv"))
        asdasd 

NameError: name 'asdasd' is not defined

In [15]:
df

,Unnamed: 0.1,Unnamed: 0,area,x,y,w,h,bbox_area,image,cell_id,is_cell
0,0,0,37780,80,1819,202,228,46056,392,0,True
1,1,1,23444,1542,1927,250,120,30000,392,1,False
2,2,2,57410,1238,1032,266,289,76874,392,2,False
3,3,3,50979,1020,1089,213,295,62835,392,3,True
4,4,4,26447,928,809,182,184,33488,392,4,False
...,...,...,...,...,...,...,...,...,...,...,...
196,196,196,29443,2670,155,267,151,40317,392,196,False
197,197,197,883,905,848,34,31,1054,392,197,False
198,198,198,14629,387,1578,142,174,24708,392,198,False
199,199,199,4920,432,1837,115,73,8395,392,199,False
